# **Calcular Costos de Tamaño**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

pd.options.display.float_format = '{:.2f}'.format

In [32]:
# Ventas categorizadas y asignación resultante
bdd_ventas = pd.read_excel("BDD_Bodegas_Categorizada_proy.xlsx")
bdd_asignaciones = pd.read_excel("resultados/Pre-Informe/datos_pmedian_p_3_manhattan_proy.xlsx")
bdd_ventas['Fecha'] = pd.to_datetime(bdd_ventas['Fecha'])
bdd_ventas['Semana'] = bdd_ventas['Fecha'].dt.strftime('%U')
bdd_ventas['Año'] = bdd_ventas['Fecha'].dt.year
bdd_ventas['Año-Sem'] = bdd_ventas['Año'].astype(str) + "-" + bdd_ventas['Semana'].astype(str)

# Comunas
bdd_comunas = pd.read_excel("BDD_Bodegas.xlsx", sheet_name=3)

bdd_asignaciones_intercambiadas = bdd_asignaciones[['Unnamed: 0', 'Bodega Asignada', 'Tiempo']]
bdd_asignaciones = bdd_asignaciones[['Unnamed: 0', 'Bodega Asignada', 'Tiempo']]

# Agrupación de ventas por cliente 
bdd_ventas_agrupadas = bdd_ventas.groupby(['ID Cliente', 'Año-Sem']).agg({"Cantidad": "sum", "Comuna Despacho": "first", 'ID Bodega Despacho': 'first', 'Categoria': 'first'}).reset_index()
bdd_ventas_agrupadas = bdd_ventas_agrupadas.merge(bdd_comunas, left_on='Comuna Despacho', right_on='Comuna')
bdd_ventas_agrupadas['Cantidad'] = bdd_ventas_agrupadas['Cantidad'].apply(lambda x: 0.01 if x == 0 else x)

# Juntamos asginaciones con demandas tanto de la bdd historica como de la proyeccion
bdd_juntadas = bdd_ventas_agrupadas.merge(bdd_asignaciones, left_on = "ID Cliente", right_on = "Unnamed: 0")

# Resultados
df_bodegas = bdd_juntadas.groupby(['Bodega Asignada', 'Año-Sem'])['Cantidad'].sum().reset_index()

In [33]:
df_bodegas

,Bodega Asignada,Año-Sem,Cantidad
0,1,2031-38,15735.06
1,1,2031-39,13258.33
2,1,2031-40,9451.81
3,1,2031-41,20811.81
4,1,2031-42,4823.50
...,...,...,...
157,8,2032-34,23954.79
158,8,2032-35,21534.75
159,8,2032-36,30903.06
160,8,2032-37,17529.91


In [34]:
promedio_por_bodega = df_bodegas.groupby('Bodega Asignada')['Cantidad'].mean()
promedio_bodegas = dict()
for i in range(len(promedio_por_bodega)):
    promedio_bodegas[promedio_por_bodega.index[i]] = promedio_por_bodega.iloc[i]

costo_tpte = 1200 # Costo por kilómetro recorrido por camión
costo_posicion = 36700 # Costo fijo de posición por producto en una bodega
distancias_Stgo_a = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}
distancias_Stgo_f = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}

In [35]:
def calcular_promedios(df_bodegas):
    promedio_por_bodega = df_bodegas.groupby('Bodega Asignada')['Cantidad'].mean()
    promedio_bodegas = dict()
    for i in range(len(promedio_por_bodega)):
        promedio_bodegas[promedio_por_bodega.index[i]] = promedio_por_bodega.iloc[i]
    return promedio_bodegas

In [36]:
len(df_bodegas)

162

In [37]:
promedio_bodegas = calcular_promedios(df_bodegas)
promedio_bodegas

{1: 8660.864345460324, 4: 57927.9005159759, 8: 19598.333184867613}

In [39]:
def calcular_promedios(df_bodegas):
    promedio_por_bodega = df_bodegas.groupby('Bodega Asignada')['Cantidad'].mean()
    promedio_bodegas = dict()
    for i in range(len(promedio_por_bodega)):
        promedio_bodegas[promedio_por_bodega.index[i]] = promedio_por_bodega.iloc[i]
    return promedio_bodegas

def tasa_ocupacion (demanda, promedio_bodegas, numero_bodega, delta):
    tamaño = promedio_bodegas[numero_bodega]*delta
    bdd = demanda[demanda["Bodega Asignada"] == numero_bodega]
    suma = 0
    for i in range(len(bdd)):
        if bdd.iloc[i, 2] >= tamaño:
            suma += 1
        else:
            suma += bdd.iloc[i, 2] / tamaño
    promedio = suma / len(bdd)
    return promedio

def semanas_sobreocupacion(demanda, promedio_bodegas, numero_bodega, delta):
    tamaño = promedio_bodegas[numero_bodega]*delta
    bdd = demanda[demanda["Bodega Asignada"] == numero_bodega]
    suma = 0
    camiones_a_pedir = 0
    for i in range(len(bdd)):
        if bdd.iloc[i, 2] > tamaño:
            suma += 1
        camiones_a_pedir += math.ceil(bdd.iloc[i,2]/tamaño)
    porcentaje = round(suma/len(bdd) * 100, 1)
    return suma, f"{porcentaje}%", camiones_a_pedir

def semanas_sobreocupacion_sim(demanda, promedio_bodegas, numero_bodega, delta):
    tamaño = promedio_bodegas[numero_bodega]*delta
    bdd = demanda[demanda["Bodega Asignada"] == numero_bodega]
    suma = 0
    camiones_a_pedir = 0
    for i in range(len(bdd)):
        if bdd.iloc[i, 2] > tamaño:
            suma += 1
        camiones_a_pedir += math.ceil(bdd.iloc[i,2]/tamaño)
    porcentaje = suma/len(bdd)
    return suma, porcentaje, camiones_a_pedir

def analisis_tamaño(demanda, promedio_bodegas, numero_bodega, delta, caso, N):
    resultados = []
    distancias_Stgo_a = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}
    distancias_Stgo_f = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}
    if caso == 'Actual':
        distancias = distancias_Stgo_a
    else:
        distancias = distancias_Stgo_f
    tamaño = promedio_bodegas[numero_bodega]*delta
    tasa = tasa_ocupacion(demanda, promedio_bodegas, numero_bodega, delta)
    semanas, porcentaje, camiones = semanas_sobreocupacion(demanda, promedio_bodegas, numero_bodega, delta)
    costo_fijo = 36700*tamaño/1000000
    costo_tpte = 1200*distancias[numero_bodega]*camiones/1000000
    costo_tpte = costo_tpte/N
    costo_total = costo_fijo + costo_tpte
    resultados.append([delta, f"{round(tasa * 100, 2)}%", tamaño, (f'{semanas} semanas, {porcentaje} del total'), costo_fijo, costo_tpte, costo_total])
    return resultados[0]

def analisis_tamaño_sim(demanda, promedio_bodegas, numero_bodega, delta, caso, N):
    resultados = []
    distancias_Stgo_a = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}
    distancias_Stgo_f = {1: 452.06, 2: 116.03, 3: 83.45, 4: 155.65, 5: 217.80, 6: 349.07, 7: 476.87, 8: 535.40, 9: 709.86, 10: 991.26}
    if caso == 'Actual':
        distancias = distancias_Stgo_a
    else:
        distancias = distancias_Stgo_f
    tamaño = promedio_bodegas[numero_bodega]*delta
    tasa = tasa_ocupacion(demanda, promedio_bodegas, numero_bodega, delta)
    semanas, porcentaje, camiones = semanas_sobreocupacion_sim(demanda, promedio_bodegas, numero_bodega, delta)
    costo_fijo = 36700*tamaño/1000000
    costo_tpte = 1200*distancias[numero_bodega]*camiones/1000000
    costo_tpte = costo_tpte/N
    costo_total = costo_fijo + costo_tpte
    resultados.append([delta, tasa, tamaño, porcentaje, costo_fijo, costo_tpte, costo_total])
    return resultados[0]

def costo_minimo(df_bodegas, promedio_bodegas, numero_bodega, caso, N):
    delta = 0
    resultado_optimo = None
    min = math.inf
    optimo = False
    while optimo == False:
        delta += 0.01
        resultado = analisis_tamaño(df_bodegas, promedio_bodegas, numero_bodega, delta, caso, N)
        costo = resultado[6]
        if costo < min:
            min = costo
            resultado_optimo = resultado
        else:
            optimo = True
    return resultado_optimo

def costo_minimo_sim(df_bodegas, promedio_bodegas, numero_bodega, caso, N):
    delta = 0
    resultado_optimo = None
    min = math.inf
    optimo = False
    while optimo == False:
        delta += 0.01
        resultado = analisis_tamaño_sim(df_bodegas, promedio_bodegas, numero_bodega, delta, caso, N)
        costo = resultado[6]
        if costo < min:
            min = costo
            resultado_optimo = resultado
        else:
            optimo = True
    return resultado_optimo

def tamaños_optimos(df_bodegas, promedio_bodegas, caso):
    N = 1
    #N = (len(df_bodegas)/3)/54
    resultados = list()
    for bodega in [1, 4, 8]:
        resultado = costo_minimo(df_bodegas, promedio_bodegas, bodega, caso, N)
        resultado.insert(0, bodega)
        resultados.append(resultado)
    df_resultados = pd.DataFrame(resultados, columns=["Bodega", "Delta", "Ocupación", "Tamaño", "Sobreocupación", "CF (MM)", "CTpte (MM)", "CT (MM)"])
    return df_resultados

def tamaños_optimos_sim(df_bodegas, promedio_bodegas, caso):
    N = 1
    #N = (len(df_bodegas)/3)/54
    resultados = list()
    for bodega in [1, 4, 8]:
        resultado = costo_minimo_sim(df_bodegas, promedio_bodegas, bodega, caso, N)
        resultado.insert(0, bodega)
        resultados.append(resultado)
    df_resultados = pd.DataFrame(resultados, columns=["Bodega", "Delta", "Ocupación", "Tamaño", "Sobreocupación", "CF (MM)", "CTpte (MM)", "CT (MM)"])
    return df_resultados

In [42]:
promedio_bodegas = calcular_promedios(df_bodegas)
costos_optimos = tamaños_optimos_sim(df_bodegas, promedio_bodegas, 'Futuro')
costos_optimos

,Bodega,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,1,0.24,1.00,2078.61,0.98,76.28,137.25,213.53
1,4,0.07,0.97,4054.95,0.96,148.82,149.24,298.05
2,8,0.20,0.96,3919.67,0.91,143.85,190.17,334.03


In [20]:
b = tamaños_optimos(df_bodegas, promedio_bodegas, 'Actual')
b

,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,0.42,92.84%,1035.02,"234 semanas, 78.0% del total",37.99,63.88,101.86
1,0.15,97.82%,1779.72,"273 semanas, 91.3% del total",65.32,77.07,142.39
2,0.39,87.18%,2000.46,"212 semanas, 70.7% del total",73.42,98.37,171.79


In [27]:
b20 = tamaños_optimos(df_bodegas, promedio_bodegas, 'Actual')
b20

,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,0.48,90.54%,1182.88,"214 semanas, 71.3% del total",43.41,68.45,111.87
1,0.17,96.85%,2017.01,"262 semanas, 87.6% del total",74.02,82.18,156.21
2,0.39,87.18%,2000.46,"212 semanas, 70.7% del total",73.42,118.04,191.46


In [8]:
c = tamaños_optimos(df_bodegas, promedio_bodegas, 'Futuro')
c

,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,0.31,99.61%,1909.98,"52 semanas, 96.3% del total",70.10,89.85,159.95
1,0.08,96.97%,4660.66,"52 semanas, 96.3% del total",171.05,160.22,331.27
2,0.23,94.92%,4398.43,"49 semanas, 90.7% del total",161.42,188.11,349.54


In [33]:
c20 = tamaños_optimos(df_bodegas, promedio_bodegas, 'Futuro')
c20

,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,0.31,99.61%,1909.98,"52 semanas, 96.3% del total",70.10,107.82,177.92
1,0.08,96.97%,4660.66,"52 semanas, 96.3% del total",171.05,192.27,363.32
2,0.23,94.92%,4398.43,"49 semanas, 90.7% del total",161.42,225.74,387.16


In [58]:
cn = tamaños_optimos_sim(df_bodegas, promedio_bodegas, 'Futuro')
cn

,Bodega,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,1,0.31,1.00,1909.98,0.96,70.10,89.85,159.95
1,4,0.08,0.97,4660.66,0.96,171.05,160.22,331.27
2,8,0.23,0.95,4398.43,0.91,161.42,188.11,349.54


In [46]:
cn = tamaños_optimos(df_bodegas, promedio_bodegas, 'Futuro')
cn

,Bodega,Delta,Ocupación,Tamaño,Sobreocupación,CF (MM),CTpte (MM),CT (MM)
0,1,0.31,99.61%,1909.98,"52 semanas, 96.3% del total",70.10,89.85,159.95
1,4,0.08,96.97%,4660.66,"52 semanas, 96.3% del total",171.05,160.22,331.27
2,8,0.23,94.92%,4398.43,"49 semanas, 90.7% del total",161.42,188.11,349.54
